In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time
from selenium import webdriver
globalUrl ="https://bonbanh.com/oto/"

In [ ]:
# Base Url sử  dụng công cụ beautifulSoup và requests để  truy xuất dữ liệu
urlDetail = "https://bonbanh.com/"

results = []

In [ ]:
# Hàm lấy thông tin bản ghi
def getInformation(data): 
    a = data.find("a")
    information = []
    tilte = a.find("div", {"class", "cb2_02"})
    # lấy thông tin hãng sản xuất
    information.append(tilte.text.split()[0])

    # lấy thông tin tên xe
    information.append(tilte.text)

    namSX = a.find("div", {"class", "cb1"}).b.text

    # lấy thông tin năm sản xuất
    information.append(namSX)

    price = a.find("div", {"class", "cb3"}).b["content"]


    contentDetail = requests.get(urlDetail + a["href"]).content
    soupDetail = BeautifulSoup(contentDetail,"html.parser")


    car_wrapper = soupDetail.find("div", {"id": "wrapper"})
    car_detail = car_wrapper.find("div", {"id": "car_detail"}).find("div", {"id": "sgg"}).find("div", {"class": "tabber"}).find("div", {"class": "tabbertab"})
    cols = car_detail.find_all("div", {"class": "col"})

    rowsCol1 = cols[0].find_all("div", {"id": "mail_parent"})

    rowsCol2 = cols[1].find_all("div", {"id": "mail_parent"})

    # lấy thông tin tình trạng xe
    if rowsCol1[1].find("div", {"class": "txt_input"}).text == "Xe đã dùng":
        information.append("used")
    else :
        information.append("new")
    # lấy thông tin loại xe
    information.append(rowsCol1[2].find("div", {"class": "txt_input"}).text.split("/")[0])

    # lấy thông tin số  km xe đã đi
    information.append(int(rowsCol1[3].find("div", {"class": "txt_input"}).text.split()[0].replace(",","")))

    # lấy thông tin số  cửa
    information.append(int(rowsCol1[6].find("div", {"class": "txt_input"}).text.split()[0]))

    # lấy thông tin số  chỗ  ngồi
    information.append(int(rowsCol1[7].find("div", {"class": "inputbox"}).text.split()[0]))

    # lấy thông tin loại nhiêu liệu
    information.append(rowsCol2[0].find("div", {"class": "txt_input"}).text.split()[0])

    # lấy thông tin lượng nhiêu liệu
    if len(rowsCol2[0].find("div", {"class": "txt_input"}).text.split()) > 1 :
        information.append(rowsCol2[0].find("div", {"class": "txt_input"}).text.split()[1])
    else :
        information.append("null")

    # lấy thông tin hộp số  xe
    information.append(rowsCol2[2].find("div", {"class": "txt_input"}).text)

    # lấy thông tin dẫn động xe
    information.append(rowsCol2[3].find("div", {"class": "txt_input"}).text)

    # lấy thông tin lượng nhiêu liệu tiêu thụ/100Km
    fuel_consumption = rowsCol2[4].find("div", {"class": "txt_input"}).text.split()[0]
    if ("L/100Km" in fuel_consumption) :
        information.append("null")
    else :
        information.append(fuel_consumption)
        
    # lấy thông tin giá xe
    information.append(price)
    print(information)
    return information

In [ ]:

for i in range(1, 2):
    time.sleep(5) 
    print(i)
    # Dữ liệu được thu thập từ các trang con của trang chính bằng cách thay đổi tham số “page” trong URL.
    content = requests.get(globalUrl + "page," + str(i)).content
    soup = BeautifulSoup(content,"html.parser")
    list = soup.find_all("li",{"class": "car-item"})
    try :
        for li in list:
            infor = getInformation(li)
            results.append(infor)
    except Exception:
        error = 'true'

In [ ]:
print(results)
# Ghi vào các bảng ghi
with open('data.csv', mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Company', 'Name', 'YearOfManufacture', 'Status', 'Body', 'DistanceUsed', 'Doors', 'Seats','FuelType', 'FuelAmount', 'GearBox', 'Drivetrain', 'FuelConsumption', 'Price'])
    for li in results:
        writer.writerow(li)